In [3]:
try:
    sc.stop()
except:
    pass
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

sc = SparkContext()
spark = SparkSession(sparkContext=sc)


# 弹性分布式数据集RDD

## RDD的概念

RDD(Resilient Distributed Datasets)，**弹性分布式数据集**，是spark中最基本的数据抽象，它代表一个不可变、可分区、里面的元素可并行计算的集合。

RDD具有数据流模型的特点：自动容错、位置感知性调度和可伸缩性。

RDD允许用户在执行多个查询时显式地将工作集缓存在内存中，后续的插叙能够重用工作集，这极大地提高了查询速度。

## RDD的属性

1. 一组分片（Partition），即数据集的基本组成单位。对于RDD来说，每个分片都会被一个计算任务处理，并决定并行计算的粒度。用户可以在创建RDD时指定RDD的分片个数，如果没有指定，那么就会采用默认值。默认值就是程序所分配到的CPU Core的数目。

2. 一个计算每个分区的函数。Spark中RDD的计算是以分片为单位的，每个RDD都会实现compute函数以达到这个目的。compute函数会对迭代器进行复合，不需要保存每次计算的结果。

3. RDD之间的依赖关系。RDD的每次转换都会生成一个新的RDD，所以RDD之间就会形成类似于流水线一样的前后依赖关系。在部分分区数据丢失时，Spark可以通过这个依赖关系重新计算丢失的分区数据，而不是对RDD的所有分区进行重新计算。

4. 一个Partitioner，即RDD的分片函数。当前Spark中实现了两种类型的分片函数，一个是基于哈希的HashPartitioner，另外一个是基于范围的RangePartitioner。只有对于于key-value的RDD，才会有Partitioner，非key-value的RDD的Parititioner的值是None。Partitioner函数不但决定了RDD本身的分片数量，也决定了parent RDD Shuffle输出时的分片数量。

5. 一个列表，存储存取每个Partition的优先位置（preferred location）。对于一个HDFS文件来说，这个列表保存的就是每个Partition所在的块的位置。按照“移动数据不如移动计算”的理念，Spark在进行任务调度的时候，会尽可能地将计算任务分配到其所要处理数据块的存储位置。

![WordCount图解RDD](../../images/WordCount图解RDD.png)

其中hello.txt

![hello.txt](../../images/hello.txt.png)


## RDD创建方式

从`pyspark.SparkContext`构建与Spark集群连接的客户端, 有两种种直接构建RDD对象的方法：

- `并行化`
- `读取文件生成`

此外也支持读取数据库生成RDD, 也支持从其他RDD转换而来.

### 并行化

`parallelize()`将python容器对象(如:`dict`,`list`,`tuple`,`set`)或可迭代对象(如:)转换为可以并行计算的分布式数据集.

In [4]:
# from a list
rdd = sc.parallelize([1,2,3])
rdd.collect()

[1, 2, 3]

In [5]:
# from a tuple
rdd = sc.parallelize(('cat', 'dog', 'fish'))
rdd.collect()

['cat', 'dog', 'fish']

In [6]:
# from a list of tuple
list_t = [('cat', 'dog', 'fish'), ('orange', 'apple')]
rdd = sc.parallelize(list_t)
rdd.collect()

[('cat', 'dog', 'fish'), ('orange', 'apple')]

In [7]:
# from a set
s = {'cat', 'dog', 'fish', 'cat', 'dog', 'dog'}
rdd = sc.parallelize(s)
rdd.collect()

['dog', 'cat', 'fish']

对于`dict`, 仅用keys构建RDD

In [8]:
# from a dict
d = {
    'a': 100,
    'b': 200,
    'c': 300
}
rdd = sc.parallelize(d)
rdd.collect()

['a', 'b', 'c']

### 读取文本文件

`textFile()`读取文本文件,返回字符串RDD,其中一个字符串元素对应文件的一行, 一般需要再执行某些**map**操作将RDD的每个元素转换为便于分析的数据结构.


In [9]:
# read a csv file
rdd = sc.textFile('../../data/mtcars.csv')
rdd.take(5)

[',mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb',
 'Mazda RX4,21,6,160,110,3.9,2.62,16.46,0,1,4,4',
 'Mazda RX4 Wag,21,6,160,110,3.9,2.875,17.02,0,1,4,4',
 'Datsun 710,22.8,4,108,93,3.85,2.32,18.61,1,1,4,1',
 'Hornet 4 Drive,21.4,6,258,110,3.08,3.215,19.44,1,0,3,1']

In [13]:
# 支持输入目录、文件通配符
rdd = sc.textFile('../../data/*.csv')
rdd.take(5)

['PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked',
 '1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.25,,S',
 '2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38,1,0,PC 17599,71.2833,C85,C',
 '3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.925,,S',
 '4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1,C123,S']